In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyMuPDF
!pip install transformers
!pip install gTTS
!pip install --upgrade googletrans==4.0.0-rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:

In [ ]:
from gtts import gTTS
import os
from googletrans import Translator

translator = Translator()

In [ ]:
import fitz  # PyMuPDF
from transformers import BartTokenizer, BartForConditionalGeneration

def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text("text")
    pdf_document.close()
    return text

def generate_summary(text):
    model_name = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    inputs = tokenizer(text, max_length=1024, return_tensors="pt", truncation=True)

    # Adjust the parameters here for longer and more comprehensive summaries
    summary_ids = model.generate(inputs["input_ids"], max_length=40000, min_length=1000, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def convert_text_to_speech(text, output_file):
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)

def main():
    pdf_path = input("Enter the path to the PDF file: ")

    pdf_text = extract_text_from_pdf(pdf_path)

    summary = generate_summary(pdf_text)
    print("\nExtractive Summary:")
    print(summary)

    # Save the summary to a text file
    output_file_path = input("Enter the path to save the summary text file: ")
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(summary)
    print(f"Summary saved to {output_file_path}")


    # conveting to audio
    input_file = output_file_path
    output_file = input("Enter the path to save the summary audio file: ")

    if not os.path.isfile(input_file):
        print(f'Error: The input file "{input_file}" does not exist.')
        return

    with open(input_file, 'r', encoding='utf-8') as f:
        research_paper_text = f.read()
    print(f"Summary audio saved to {output_file}")

    convert_text_to_speech(research_paper_text, output_file)

if __name__ == "__main__":
    main()
